In [1]:
from src.data_local import MergedDataPreprocessing
from cashe import get_cashed_input

In [2]:
# !pip install tensorflow --upgrade


## HJH Dataset
loaded on 23/5/2024

In [3]:
df_original = get_cashed_input()

In [4]:
df_original['CREATION_DATE'].sort_values(ascending=False)

388640   2024-04-30 12:13:09
388639   2024-04-30 12:13:09
388633   2024-04-30 12:13:09
388634   2024-04-30 12:13:09
388635   2024-04-30 12:13:09
                 ...        
397544   2024-01-01 09:58:15
397545   2024-01-01 09:58:15
397546   2024-01-01 09:58:15
397547   2024-01-01 09:58:15
397549   2024-01-01 09:58:15
Name: CREATION_DATE, Length: 488833, dtype: datetime64[ns]

In [5]:
df_original.VISIT_ID.value_counts()

VISIT_ID
220020    1819
202392    1529
203244    1529
201986    1529
202394    1529
          ... 
198125       1
228482       1
210011       1
228691       1
231136       1
Name: count, Length: 42771, dtype: int64

In [6]:
preprocessing = MergedDataPreprocessing(df_original)

In [7]:
for col in df_original.columns:
    print(col,":",df_original[col].dtype)

CREATED_BY_USER : int64
CREATION_DATE : datetime64[ns]
VISIT_ID : int64
NET_WITH_VAT : float64
SERVICE_DESCRIPTION : object
QTY : float64
LINE_CLAIMED_AMOUNT : float64
LINE_CLAIMED_AMOUNT_SAR : float64
CO_PAY : float64
CO_INSURANCE : float64
LINE_ITEM_DISCOUNT : float64
NET_AMOUNT : float64
NET_VAT_AMOUNT : float64
PATIENT_VAT_AMOUNT : float64
VAT_PERCENTAGE : float64
TREATMENT_TYPE_INDICATOR : int64
SERVICE_TYPE : object
DURATION : float64
QTY_STOCKED_UOM : int64
UNIT_PRICE : float64
OASIS_IOS_DESCRIPTION : object
SEQUENCE_NO : int64
UNIT_PRICE_STOCKED_UOM : float64
UNIT_PRICE_NET : float64
DISCOUNT_PERCENTAGE : float64
NOTES : object
OUTCOME : object
APPROVED_QUNATITY : float64
REQUEST_DATE : datetime64[ns]
VISIT_NO : object
VISIT_DATE : datetime64[ns]
EMERGENCY_INDICATOR : object
PROVIDER_DEPARTMENT_CODE : int64
PROVIDER_DEPARTMENT : object
DOCTOR_SPECIALTY_CODE : float64
DOCTOR_CODE : int64
PATIENT_NO : int64
EPISODE_NO : int64
PATIENT_ID : int64
MEMBERSHIP_NO : object
PATIENT_ID_T

In [8]:
df_original.isna().sum()

CREATED_BY_USER            0
CREATION_DATE              0
VISIT_ID                   0
NET_WITH_VAT               0
SERVICE_DESCRIPTION        0
                       ...  
BATCH_PERIOD_START         0
BATCH_PERIOD_END           0
Episode_Key            31109
ICD10                  31109
Diagnosis_Key          31109
Length: 96, dtype: int64

In [9]:
df_original.SUBMIT_CLAIM_OUTCOME.value_counts()

SUBMIT_CLAIM_OUTCOME
PENDED    241830
QUEUED     70352
ERROR      64160
Name: count, dtype: int64

In [10]:
df_prep = preprocessing.columns_prep()

C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:203: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:208: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)
C:\AI_Projects\Claims_Rejection_2024\Claims_Rejection\src\data_local.py:214: UserWarning: Pandas doesn't allow columns to be created via a new attribute nam

In [11]:
df_prep

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key,PatientAgeRange
0,9521,2024-04-04 22:57:57,230089,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,...,0,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31,1_929362-34,Z36,1_10198603,6
1,9521,2024-04-04 23:02:21,230117,202.40,TSH,1.0,220.00,220.00,0.00,220.00,...,0,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31,1_1083147-3,R22.1,1_10171516,5
2,9521,2024-04-04 23:08:38,230160,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,...,0,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31,1_1339075-6,R53,1_1111232092,6
3,9521,2024-04-04 22:26:08,229872,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,...,0,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31,1_656928-60,R10,1_1111231597,6
4,9521,2024-04-04 22:34:22,229913,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,...,0,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31,1_1019628-25,R00.1,1_10171441,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,...,0,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31,1_1348956-2,I24.9,1_10163946,8
488829,10962,2024-01-28 01:01:05,204377,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,...,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,1_1349595-2,R07.4,1_110217036,7
488830,10962,2024-01-28 01:01:05,204377,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,...,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,1_1349595-2,R07.4,1_110217036,7
488831,10962,2024-01-28 01:01:05,204377,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,...,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,1_1349595-2,R07.4,1_110217036,7


In [12]:
df_train, df_test = preprocessing.train_test_split(test_size=0.2,random_state=6)

In [13]:
len(df_test)/ len(df_train)

0.2531547822058057

In [14]:
ls1 = list(df_train.sort_values(by='VISIT_ID')['VISIT_ID'])
ls2 = list(df_test.sort_values(by='VISIT_ID')['VISIT_ID'])
intersection = list(set(ls1) & set(ls2))
intersection ## to find the repeated transaction id between them

[]

In [15]:
len(df_train) / len(df_test)

3.9501525165226234

In [16]:
df_train.to_parquet('data/HJH/prq/train.parquet')
df_test.to_parquet('data/HJH/prq/test.parquet')